In [1]:
from network_utils_hybrid import *
import random
import numpy as np
import time
import pyomo.environ as pe



In [3]:
Nrep = 100
num_gates_list = np.arange(10,41,10)
num_ToR = 4
n = 6 # must be even, starts from 4

specs = {
    "num_sw_ports": n,
    "num_ToR" : num_ToR,
    "qs_per_node" : 10,
    "bandwidth" : 2,
    "num_bsm_ir" : 2,
    "num_bsm_tel" : 12,
    "num_pd" : 1, # inactive
    "num_laser" : 1, # inactive
    "num_bs" : 1, # inactive
    "num_es" : 1 # inactive
}

G, vertex_list = clos_hybrid(specs)
edge_switches, node_list, node_qubit_list =  vertex_list


In [4]:
num_gates = 100
query_seq, gate_mul_seq = parallel_circuit_gen(node_list, specs["qs_per_node"], num_gates)
print(query_seq)


NameError: name 'parallel_circuit_gen' is not defined

In [3]:
num_gates = 100
query_seq, gate_mul_seq = parallel_circuit_gen(node_list, specs["qs_per_node"], num_gates)
print(query_seq)

sw_tel_bsm = {}
sw_nir_bsm = {}
for node in G.nodes():
    if not node in (list(node_list)+node_qubit_list):
        # print(G.nodes[node]["BSM_tel"])
        if G.nodes[node]["BSM_tel"]>0:
            sw_tel_bsm[node] = 0#G.nodes[node]["BSM_tel"]
        if G.nodes[node]["BSM_ir"]>0:
            sw_nir_bsm[node] = 0# G.nodes[node]["BSM_nir"]

print(sw_tel_bsm)
print(sw_nir_bsm)


        
# print(num_paths)
# print(path_list["ir"])


[[('25,5', '28,0'), ('22,5', '18,2'), ('49,7', '41,1'), ('27,9', '21,9'), ('18,6', '39,6'), ('28,7', '48,7'), ('24,4', '26,6'), ('20,2', '33,9'), ('51,8', '29,0'), ('20,8', '39,7'), ('44,8', '33,5'), ('50,7', '32,9'), ('39,8', '35,3'), ('40,5', '32,7'), ('40,3', '32,6'), ('27,7', '21,2'), ('38,6', '36,5'), ('52,2', '33,4'), ('48,4', '43,2'), ('26,3', '44,5'), ('42,0', '19,5'), ('30,3', '52,8'), ('49,2', '43,6'), ('18,1', '43,4'), ('45,7', '32,4'), ('30,9', '26,8'), ('25,1', '53,3'), ('50,0', '29,8'), ('36,2', '39,4'), ('19,4', '32,2'), ('40,2', '23,8'), ('32,8', '33,0'), ('46,7', '42,4'), ('29,5', '40,9'), ('47,6', '45,3'), ('52,1', '28,1'), ('44,3', '46,9'), ('26,5', '31,3'), ('52,6', '25,6'), ('24,7', '46,6'), ('35,7', '28,5'), ('44,4', '49,0'), ('38,4', '37,2'), ('40,1', '18,0'), ('36,9', '52,5'), ('19,8', '32,3'), ('29,1', '35,6'), ('28,6', '36,3'), ('34,9', '24,1'), ('34,2', '53,2'), ('26,0', '37,9'), ('30,4', '24,2'), ('30,0', '35,8'), ('29,6', '25,3'), ('31,7', '38,5'), ('27,3',

In [2]:
num_gates = 100
query_seq, gate_mul_seq = parallel_circuit_gen(node_list, specs["qs_per_node"], num_gates)
print(query_seq)

sw_tel_bsm = {}
sw_nir_bsm = {}
for node in G.nodes():
    if not node in (list(node_list)+node_qubit_list):
        # print(G.nodes[node]["BSM_tel"])
        if G.nodes[node]["BSM_tel"]>0:
            sw_tel_bsm[node] = 0#G.nodes[node]["BSM_tel"]
        if G.nodes[node]["BSM_ir"]>0:
            sw_nir_bsm[node] = 0# G.nodes[node]["BSM_nir"]

# print(sw_tel_bsm)
# print(sw_nir_bsm)


# cplex implementation
from docplex.mp.model import Model

for i_q, gate_seq in enumerate(query_seq[:1]):
    gate_seq_iter = gate_seq[:]
    gate_mul_seq_iter = gate_mul_seq[i_q]
    sw_events = []
    while len(gate_seq_iter)>0:

        end_nodes = {"tel" : {}, "ir" : {}}
        num_paths = {"tel" : 0, "ir" : 0}
        num_paths_gate = []
        path_list = {"tel": [], "ir": []}
        bsm_nodes = []
        # print(i_q, gate_seq_iter)
        for i_g, g in enumerate(gate_seq_iter):
            # for link in range(gate_mul_seq_iter[g]):
            n0 = g[0]
            n1 = g[1]
            paths = nx.all_shortest_paths(G, n0, n1, weight=None)
            path_count = 0
            for shortestpath in paths:
                if len(shortestpath)> 5 :
                    tel_ir = "tel"
                    for node in shortestpath:
                        if node in sw_tel_bsm.keys():
                            bsm_nodes.append(node)
                            end_nodes["tel"][num_paths["tel"]] = g
                            path_list["tel"].append(shortestpath[1:-1])
                            num_paths["tel"] += 1
                            path_count += 1
                else:
                    tel_ir = "ir"
                    end_nodes["ir"][num_paths["ir"]] = g
                    num_paths["ir"] += 1
                    path_list["ir"].append(shortestpath[1:-1])
            if tel_ir == "tel":
                num_paths_gate.append(path_count)
        mdl = Model(name='routing')
        x  = {(i): mdl.binary_var(name=f"x_{i}") for i in range(num_paths["tel"])}
        y  = {(i): mdl.binary_var(name=f"y_{i}") for i in range(num_paths["ir"])}

        # counter_paths = 0
        # for i_g, g in enumerate(gate_seq_iter):
        #     n0 = g[0]
        #     n1 = g[1]
        #     paths = nx.all_shortest_paths(G, n0, n1, weight=None)
        #     one_path_per_gate = 0
        #     for shortestpath in paths:
        #         if len(shortestpath)<= 5 :
        #             # ir path is unique
        #             tel_ir = "ir"
        #             break
        #         else:
        #             # print(path_list["tel"][counter_paths])
        #             one_path_per_gate += x[counter_paths]
        #             counter_paths += 1
        #         tel_ir = "tel"
        #     if tel_ir == "tel":
        #         # print(one_path_per_gate)
        #         mdl.add_constraint(  one_path_per_gate <= 1 ) 

        idx1 = 0
        for num_paths_per_gate in num_paths_gate:
            one_path_per_gate = 0
            idx2 = idx1+num_paths_per_gate
            for i_p in range(idx1,idx2):
                one_path_per_gate += x[i_p]
            mdl.add_constraint(  one_path_per_gate <= 1 )
            idx1 = idx2

        comm_qs = {node: 0 for node in node_list}
        sw_tel_bsm = {key: 0 for key in sw_tel_bsm.keys()}
        sw_nir_bsm = {key: 0 for key in sw_nir_bsm.keys()}

        for i_p, path in enumerate(path_list["ir"]):
            for node in path:
                if node in sw_nir_bsm.keys():
                    # print(node, path)
                    sw_nir_bsm[node] += y[i_p]
                # sw_nir_bsm[node]
                if node in node_list:
                    comm_qs[node] += y[i_p]

        for node in sw_nir_bsm.keys():
            mdl.add_constraint( sw_nir_bsm[node] <= G.nodes[node]["BSM_ir"])

        for i_p, path in enumerate(path_list["tel"]):
            sw_tel_bsm[bsm_nodes[i_p]] += x[i_p] 
            for node in path:
                if node in node_list:
                    comm_qs[node] += x[i_p]
                
        for node in sw_tel_bsm.keys():
            mdl.add_constraint( sw_tel_bsm[node] <= G.nodes[node]["BSM_tel"])

        for node in node_list:
            # print(G.nodes[node]["comm"])
            mdl.add_constraint( comm_qs[node] <= G.nodes[node]["comm"])


        obj = 0
        for i in range(num_paths["tel"]):
            obj += x[i]
        coef = 1.0
        for i in range(num_paths["ir"]):
            obj += coef* y[i]
        mdl.maximize(obj)

        mdl.maximize(obj)
        # del solution
        solution = mdl.solve()#log_output=True,)

        x_opt = np.array([solution[x[i]] for i in range(num_paths["tel"]) ])
        y_opt = np.array([solution[y[i]] for i in range(num_paths["ir"]) ])
        sw_events.append([y_opt.sum().astype(int), x_opt.sum().astype(int)])
        
        exec_gates_ir = [end_nodes["ir"][g] for g in np.argwhere(y_opt>0)[:,0]]
        exec_gates_tel = [end_nodes["tel"][g] for g in np.argwhere(x_opt>0)[:,0]]
        # print(exec_gates_ir)
        # print(exec_gates_tel)
        gate_seq_iter = list(set(gate_seq_iter)-set(exec_gates_ir+exec_gates_tel))
        # exec_gates_ir = [path_list["ir"][g] for g in np.argwhere(y_opt>0)[:,0]]
        # exec_gates_tel = [path_list["tel"][g] for g in np.argwhere(x_opt>0)[:,0]]
        # print(exec_gates_ir)
        # print(exec_gates_tel)
        # break

print(sw_events)
print(len(sw_events))
# x_opt = np.zeros(num_paths["tel"])
# for i in range(num_paths["tel"]):
#     x_opt[i] = mdl.x[i].value
# print(x_opt)

# y_opt = np.zeros(num_paths["ir"])
# for i in range(num_paths["ir"]):
#     y_opt[i] = model.y[i].value
# # print(y_opt)
# for v in mdl.iter_binary_vars():
#     print(v," = ",v.solution_value)


NameError: name 'parallel_circuit_gen' is not defined

In [5]:
# x_opt = np.array([solution[x[i]] for i in range(num_paths["tel"]) ])
# y_opt = np.array([solution[y[i]] for i in range(num_paths["ir"]) ])
# print(y_opt.sum(), x_opt.sum())

switch_seq = network_latency_multiqubit_hybrid(G, vertex_list, query_seq[:1], gate_mul_seq[:1])
print(switch_seq[0])
print(len(switch_seq[0]))


[[3, 24], [3, 21], [1, 12], [0, 1]]
4


In [ ]:
# num_gates = 100
# query_seq, gate_mul_seq = parallel_circuit_gen(node_list, specs["qs_per_node"], num_gates)
# print(query_seq)

sw_tel_bsm = {}
sw_nir_bsm = {}
for node in G.nodes():
    if not node in (list(node_list)+node_qubit_list):
        # print(G.nodes[node]["BSM_tel"])
        if G.nodes[node]["BSM_tel"]>0:
            sw_tel_bsm[node] = 0#G.nodes[node]["BSM_tel"]
        if G.nodes[node]["BSM_ir"]>0:
            sw_nir_bsm[node] = 0# G.nodes[node]["BSM_nir"]

print(sw_tel_bsm)
print(sw_nir_bsm)


# cplex implementation
from docplex.mp.model import Model

for i_q, gate_seq in enumerate(query_seq[:1]):
    gate_seq_iter = gate_seq[:]
    gate_mul_seq_iter = gate_mul_seq[i_q]
    while len(gate_seq_iter)>0:

        end_nodes = {"tel" : {}, "ir" : {}}
        num_paths = {"tel" : 0, "ir" : 0}
        path_list = {"tel": [], "ir": []}
        # print(i_q, gate_seq_iter)
        for i_g, g in enumerate(gate_seq_iter):
            # for link in range(gate_mul_seq_iter[g]):
            n0 = g[0]
            n1 = g[1]
            paths = nx.all_shortest_paths(G, n0, n1, weight=None)
            for shortestpath in paths:
                if len(shortestpath)> 5 :
                    end_nodes["tel"][num_paths["tel"]] = g
                    num_paths["tel"] += 1
                    # path_list["tel"].append(shortestpath[1:-1])
                    path_list["tel"].append(shortestpath)
                else:
                    end_nodes["ir"][num_paths["ir"]] = g
                    num_paths["ir"] += 1
                    # path_list["ir"].append(shortestpath[1:-1])
                    path_list["ir"].append(shortestpath)
                # break
                
        mdl = Model(name='routing')
        x  = {(i): mdl.binary_var(name=f"x_{i}") for i in range(num_paths["tel"])}
        y  = {(i): mdl.binary_var(name=f"y_{i}") for i in range(num_paths["ir"])}

        counter_paths = 0

        for i_g, g in enumerate(gate_seq_iter):
            n0 = g[0]
            n1 = g[1]
            paths = nx.all_shortest_paths(G, n0, n1, weight=None)
            one_path_per_gate = 0
            for shortestpath in paths:
                if len(shortestpath)<= 5 :
                    # ir path is unique
                    tel_ir = "ir"
                    break
                else:
                    # print(path_list["tel"][counter_paths])
                    one_path_per_gate += x[counter_paths]
                    counter_paths += 1
                tel_ir = "tel"
            if tel_ir == "tel":
                # print(one_path_per_gate)
                mdl.add_constraint(  one_path_per_gate <= 1 ) 

        comm_qs = {node: 0 for node in node_list}
        sw_tel_bsm = {key: 0 for key in sw_tel_bsm.keys()}
        sw_nir_bsm = {key: 0 for key in sw_nir_bsm.keys()}

        for i_p, path in enumerate(path_list["ir"]):
            for node in path:
                if node in sw_nir_bsm.keys():
                    # print(node, path)
                    sw_nir_bsm[node] += y[i_p]
                # sw_nir_bsm[node]
                if node in node_list:
                    comm_qs[node] += y[i_p]

        for node in sw_nir_bsm.keys():
            mdl.add_constraint( sw_nir_bsm[node] <= G.nodes[node]["BSM_ir"])

        for i_p, path in enumerate(path_list["tel"]):
            for node in path:
                if node in sw_tel_bsm.keys():
                    # print(node, path)
                    sw_tel_bsm[node] += x[i_p]
                # sw_nir_bsm[node]
                if node in node_list:
                    comm_qs[node] += x[i_p]
                
        for node in sw_tel_bsm.keys():
            mdl.add_constraint( sw_tel_bsm[node] <= G.nodes[node]["BSM_tel"])

        for node in node_list:
            # print(G.nodes[node]["comm"])
            mdl.add_constraint( comm_qs[node] <= G.nodes[node]["comm"])


        obj = 0
        for i in range(num_paths["tel"]):
            obj += x[i]
        coef = 1.0
        for i in range(num_paths["ir"]):
            obj += coef* y[i]
        # mdl.maximize(obj)

        mdl.maximize(obj)
        # del solution
        solution = mdl.solve()#log_output=True,)

        x_opt = np.array([solution[x[i]] for i in range(num_paths["tel"]) ])
        y_opt = np.array([solution[y[i]] for i in range(num_paths["ir"]) ])
        print(y_opt.sum(), x_opt.sum())
        exec_gates_ir = [end_nodes["ir"][g] for g in np.argwhere(y_opt>0)[:,0]]
        exec_gates_tel = [end_nodes["tel"][g] for g in np.argwhere(x_opt>0)[:,0]]
        # print(exec_gates_ir)
        # print(exec_gates_tel)
        gate_seq_iter = list(set(gate_seq_iter)-set(exec_gates_ir+exec_gates_tel))
        exec_gates_ir = [path_list["ir"][g] for g in np.argwhere(y_opt>0)[:,0]]
        exec_gates_tel = [path_list["tel"][g] for g in np.argwhere(x_opt>0)[:,0]]
        print(exec_gates_ir)
        print(exec_gates_tel)
        # break

# x_opt = np.zeros(num_paths["tel"])
# for i in range(num_paths["tel"]):
#     x_opt[i] = mdl.x[i].value
# print(x_opt)

# y_opt = np.zeros(num_paths["ir"])
# for i in range(num_paths["ir"]):
#     y_opt[i] = model.y[i].value
# # print(y_opt)
# for v in mdl.iter_binary_vars():
#     print(v," = ",v.solution_value)


In [28]:
from cplex import Cplex

# Create an instance of the CPLEX problem
problem = Cplex()

# Set the objective to maximization
problem.objective.set_sense(problem.objective.sense.maximize)

# Add binary variables x0, x1, x2
variable_names = ["x0", "x1", "x2"]
problem.variables.add(obj=[1.0, 1.0, 1.0], types=[problem.variables.type.binary] * 3, names=variable_names)

# Add the constraint: x0 + x1 <= 1
constraint = [["x0", "x1"], [1.0, 1.0]]
problem.linear_constraints.add(lin_expr=[constraint], senses=["L"], rhs=[1.0], names=["c1"])

# Solve the problem
problem.solve()

# Print the solution
solution_values = problem.solution.get_values()
objective_value = problem.solution.get_objective_value()

print("Solution values:", solution_values)
print("Objective value:", objective_value)


Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 1 rows and 3 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.00 ticks)

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.00 ticks)
Parallel b&c, 32 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.00 ticks)
Solution values: [1.0, 0.0, 1.0]
Objective value: 2.0


In [27]:
# pyomo implementation

model = pe.ConcreteModel()
model.constraints = pe.ConstraintList()
model.x = pe.Var(range(num_paths["tel"]) , domain=pe.Binary, initialize=1)
model.y = pe.Var(range(num_paths["ir"]), domain=pe.Binary, initialize=1)

counter_paths = 0

for i_q, gate_seq in enumerate(query_seq[:1]):    
    gate_seq_iter = gate_seq[:]
    gate_mul_seq_iter = gate_mul_seq[i_q]
    for i_g, g in enumerate(gate_seq_iter):
        n0 = g[0]
        n1 = g[1]
        paths = nx.all_shortest_paths(G, n0, n1, weight=None)
        one_path_per_gate = 0
        for shortestpath in paths:
            if len(shortestpath)<= 5 :
                # ir path is unique
                tel_ir = "ir"
                break
            else:
                print(path_list["tel"][counter_paths])
                one_path_per_gate += model.x[counter_paths]
                counter_paths += 1
            tel_ir = "tel"
        # if tel_ir == "tel":
        #     print(one_path_per_gate)
        #     model.constraints.add(  one_path_per_gate <= 1 ) 

comm_qs = {node: 0 for node in node_list}

for i_p, path in enumerate(path_list["ir"]):
    for node in path:
        if node in sw_nir_bsm.keys():
            # print(node, path)
            sw_nir_bsm[node] += model.y[i_p]
        # sw_nir_bsm[node]
        if node in node_list:
            comm_qs[node] += model.y[i_p]
        
# for node in sw_nir_bsm.keys():
#     model.constraints.add( sw_nir_bsm[node] <= G.nodes[node]["BSM_ir"])

for i_p, path in enumerate(path_list["tel"]):
    for node in path:
        if node in sw_tel_bsm.keys():
            # print(node, path)
            sw_tel_bsm[node] += model.x[i_p]
        # sw_nir_bsm[node]
        if node in node_list:
            comm_qs[node] += model.x[i_p]
        
# for node in sw_tel_bsm.keys():
#     model.constraints.add( sw_tel_bsm[node] <= G.nodes[node]["BSM_tel"])

# for node in node_list:
#     # print(G.nodes[node]["comm"])
#     model.constraints.add( comm_qs[node] <= G.nodes[node]["comm"])

model.objective = pe.Objective(expr= sum(model.x), sense=pe.maximize)
opt = pe.SolverFactory('cplex_direct')
results = opt.solve(model)
results.write()

x_opt = np.zeros(num_paths["tel"])
for i in range(num_paths["tel"]):
    x_opt[i] = model.x[i].value
print(x_opt)

y_opt = np.zeros(num_paths["ir"])
for i in range(num_paths["ir"]):
    y_opt[i] = model.y[i].value
print(y_opt)

{2: 0, 3: 0, 4: 0, 5: 0}
{6: 0, 7: 0, 8: 0, 9: 0}
0 [('14,1', '16,5'), ('13,8', '16,1')]
{'tel': 10, 'ir': 0}
[14, 8, 4, 9, 16]
[14, 8, 5, 9, 16]
[13, 7, 2, 0, 4, 9, 16]
[13, 7, 3, 0, 4, 9, 16]
[13, 7, 2, 1, 4, 9, 16]
[13, 7, 3, 1, 4, 9, 16]
[13, 7, 2, 0, 5, 9, 16]
[13, 7, 3, 0, 5, 9, 16]
[13, 7, 2, 1, 5, 9, 16]
[13, 7, 3, 1, 5, 9, 16]
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: 
  Lower bound: 45.0
  Upper bound: 45.0
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 10
  Number of binary variables: 10
  Number of integer variables: 0
  Number of continuous variables: 0
  Number of nonzeros: None
  Sense: maximize
# ----------------------------------------------

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [92]:
# sanity check

problem_size = 3
model = pe.ConcreteModel()
model.x = pe.Var(range(problem_size) , domain=pe.Binary, initialize=0)
# model.constraints = pe.ConstraintList()
# model.constraints.add( model.x[0]+  model.x[1] <= 2)
model.c1 = pe.Constraint( expr = (model.x[0]+  model.x[1] <= 1) )

model.objective = pe.Objective(expr= sum(model.x), sense=pe.maximize)
opt = pe.SolverFactory('cplex_direct', tee=True)
# opt = pe.SolverFactory('cplex', executable="/nfshome/hassan/cplex/2210/cplex/bin/x86-64_linux/cplex")
model.display()
results = opt.solve(model)
results.write()
# # # model.objective.display()
# # print(pe.value(model.objective))
# # # model.display()
# # # model.pprint()

x_opt = np.zeros(problem_size)
for i in range(problem_size):
    x_opt[i] = pe.value(model.x[i])
print(x_opt)

Model unknown

  Variables:
    x : Size=3, Index={0, 1, 2}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :     0 :     1 : False : False : Binary
          1 :     0 :     0 :     1 : False : False : Binary
          2 :     0 :     0 :     1 : False : False : Binary

  Objectives:
    objective : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :     3

  Constraints:
    c1 : Size=1
        Key  : Lower : Body : Upper
        None :  None :    0 :   1.0
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: 
  Lower bound: 3.0
  Upper bound: 3.0
  Number of objectives: 1
  Number of constraints: 1
  Number of variables: 3
  Num

In [88]:
for res in results.items():
    print(results['Solver'])


- Name: CPLEX 22.1.0.0
  Status: ok
  Wallclock time: 0.005804777145385742
  Termination condition: optimal


- Name: CPLEX 22.1.0.0
  Status: ok
  Wallclock time: 0.005804777145385742
  Termination condition: optimal


- Name: CPLEX 22.1.0.0
  Status: ok
  Wallclock time: 0.005804777145385742
  Termination condition: optimal



In [45]:
opt = pe.SolverFactory('cplex_direct')
results = opt.solve(model)
results.write()


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: 
  Lower bound: 3.0
  Upper bound: 3.0
  Number of objectives: 1
  Number of constraints: 1
  Number of variables: 3
  Number of binary variables: 3
  Number of integer variables: 0
  Number of continuous variables: 0
  Number of nonzeros: None
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Name: CPLEX 22.1.0.0
  Status: ok
  Wallclock time: 0.005006313323974609
  Termination condition: optimal
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------